In [1]:

import scanpy as sc
import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import networkx as nx
import torch
import sklearn.preprocessing
import episcanpy as epi
import sys
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
# import networkx as nx # No longer needed for training script itself
import torch


adata_ATAC=sc.read("/data/cabins/sccancer/hhy/TI/adult_human_kidney_ATAC.h5ad")
adata_RNA= sc.read("/data/cabins/sccancer/hhy/TI/adult_human_kidney_RNA.h5ad")

In [2]:
adata_ATAC_copy = adata_ATAC.copy()
adata_RNA_copy = adata_RNA.copy()

In [3]:
adata_ATAC = adata_ATAC_copy 
adata_RNA = adata_RNA_copy

In [4]:
adata_ATAC

AnnData object with n_obs × n_vars = 27034 × 19520
    obs: 'mapped_reference_assembly', 'alignment_software', 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'donor_living_at_sample_collection', 'organism_ontology_term_id', 'sample_uuid', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'sample_derivation_process', 'sample_source', 'suspension_derivation_process', 'suspension_dissociation_reagent', 'suspension_uuid', 'suspension_type', 'library_uuid', 'assay_ontology_term_id', 'sequencing_platform', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'disease_ontology_term_id', 'reported_diseases', 'sex_ontology_term_id', 'seurat_clusters', 'nCount_RNA', 'nFeature_RNA', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length

In [5]:
adata_ATAC.X.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
adata_RNA

AnnData object with n_obs × n_vars = 19985 × 33145
    obs: 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sample_uuid', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'suspension_uuid', 'suspension_type', 'library_uuid', 'assay_ontology_term_id', 'mapped_reference_annotation', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'disease_ontology_term_id', 'reported_diseases', 'sex_ontology_term_id', 'nCount_SCT', 'nFeature_SCT', 'percent.mt', 'percent.rpl', 'percent.rps', 'seurat_clusters', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type'
    uns: 'citation', 'default_embedding', 'schema_reference', 'schema_version', 'title'
    obsm: 'X_harmony', 'X_pca', 'X_umap'

In [7]:
adata_RNA.X.A

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.6931472, 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 1.0986123, 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [8]:
adata_ATAC.obs["cell_type"].value_counts()

cell_type
kidney loop of Henle thick ascending limb epithelial cell    7775
kidney proximal convoluted tubule epithelial cell            6384
kidney proximal straight tubule epithelial cell              4146
kidney distal convoluted tubule epithelial cell              2781
renal principal cell                                         1291
kidney connecting tubule epithelial cell                      884
kidney capillary endothelial cell                             759
epithelial cell of proximal tubule                            690
renal beta-intercalated cell                                  620
renal alpha-intercalated cell                                 613
parietal epithelial cell                                      402
connective tissue cell                                        363
leukocyte                                                     193
podocyte                                                      133
Name: count, dtype: int64

In [9]:
adata_ATAC.obs["development_stage"].value_counts()

development_stage
61-year-old stage    7381
52-year-old stage    6135
50-year-old stage    5280
54-year-old stage    4166
62-year-old stage    4072
Name: count, dtype: int64

In [10]:
adata_RNA.obs["cell_type"].value_counts()

cell_type
epithelial cell of proximal tubule                           5485
kidney loop of Henle thick ascending limb epithelial cell    4435
kidney distal convoluted tubule epithelial cell              3250
kidney connecting tubule epithelial cell                     1805
renal alpha-intercalated cell                                1107
renal principal cell                                         1022
kidney capillary endothelial cell                            1008
parietal epithelial cell                                      552
podocyte                                                      463
renal beta-intercalated cell                                  349
mesangial cell                                                239
fibroblast                                                    207
leukocyte                                                      63
Name: count, dtype: int64

In [11]:
adata_RNA.obs["development_stage"].value_counts()

development_stage
61-year-old stage    5464
54-year-old stage    4114
52-year-old stage    3804
50-year-old stage    3683
62-year-old stage    2920
Name: count, dtype: int64

In [12]:
adata_ATAC = adata_ATAC[adata_ATAC.obs["cell_type"] == 'kidney loop of Henle thick ascending limb epithelial cell',:]
samples_to_keep = ['50-year-old stage', '52-year-old stage', '54-year-old stage', '61-year-old stage','62-year-old stage']
mask = adata_ATAC.obs['development_stage'].isin(samples_to_keep)
adata_ATAC = adata_ATAC[mask].copy()

In [13]:
adata_RNA = adata_RNA[adata_RNA.obs["cell_type"] == 'kidney loop of Henle thick ascending limb epithelial cell',:]
samples_to_keep = ['50-year-old stage', '52-year-old stage', '54-year-old stage', '61-year-old stage','62-year-old stage']
mask = adata_RNA.obs['development_stage'].isin(samples_to_keep)
adata_RNA = adata_RNA[mask].copy()

In [29]:
adata_RNA.obs["cell_type"]

AAACCTGCAAGCGCTC-1    kidney loop of Henle thick ascending limb epit...
AAACCTGCACCAGATT-1    kidney loop of Henle thick ascending limb epit...
AAACCTGTCCCATTAT-1    kidney loop of Henle thick ascending limb epit...
AAACCTGTCGCCTGAG-1    kidney loop of Henle thick ascending limb epit...
AAACGGGAGCCCAATT-1    kidney loop of Henle thick ascending limb epit...
                                            ...                        
TTGCCGTGTACTCAAC-5    kidney loop of Henle thick ascending limb epit...
TTGCCGTTCGTGGGAA-5    kidney loop of Henle thick ascending limb epit...
TTTATGCTCCAACCAA-5    kidney loop of Henle thick ascending limb epit...
TTTCCTCAGAGGTTGC-5    kidney loop of Henle thick ascending limb epit...
TTTGCGCAGATGTGGC-5    kidney loop of Henle thick ascending limb epit...
Name: cell_type, Length: 4435, dtype: category
Categories (1, object): ['kidney loop of Henle thick ascending limb epi...]

In [14]:
adata_ATAC.obs['development_stage'].value_counts()

development_stage
52-year-old stage    2287
50-year-old stage    1966
61-year-old stage    1855
54-year-old stage    1214
62-year-old stage     453
Name: count, dtype: int64

In [15]:
adata_RNA.obs['development_stage'].value_counts()

development_stage
50-year-old stage    1995
54-year-old stage    1017
61-year-old stage     628
52-year-old stage     402
62-year-old stage     393
Name: count, dtype: int64

In [16]:
replacement_map = {
    '50-year-old stage': '50',
    '52-year-old stage': '52',
    '54-year-old stage': '54',
    '61-year-old stage': '61',
    '62-year-old stage': '62',   
}
adata_ATAC.obs['age'] = adata_ATAC.obs['development_stage'].replace(replacement_map)
adata_RNA.obs['age'] = adata_RNA.obs['development_stage'].replace(replacement_map)

In [17]:
sc.pp.filter_cells(adata_RNA, min_genes=100)
sc.pp.filter_genes(adata_RNA, min_cells=3)
# Saving count data
adata_RNA.layers["counts"] = adata_RNA.X.copy()
# Normalizing to median total counts
sc.pp.normalize_total(adata_RNA)
# Logarithmize the data
sc.pp.log1p(adata_RNA)
sc.pp.highly_variable_genes(adata_RNA, n_top_genes=2000)
adata_RNA = adata_RNA[:,adata_RNA.var['highly_variable'] ==True]
fpeak=0.03
epi.pp.binarize(adata_ATAC)
epi.pp.filter_features(adata_ATAC, min_cells=np.ceil(adata_ATAC.shape[0])*fpeak)

In [18]:
adata_RNA

View of AnnData object with n_obs × n_vars = 4435 × 2000
    obs: 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sample_uuid', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'suspension_uuid', 'suspension_type', 'library_uuid', 'assay_ontology_term_id', 'mapped_reference_annotation', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'disease_ontology_term_id', 'reported_diseases', 'sex_ontology_term_id', 'nCount_SCT', 'nFeature_SCT', 'percent.mt', 'percent.rpl', 'percent.rps', 'seurat_clusters', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'age', 'n_genes'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'citation', 'default_embe

In [19]:
# -*- coding: utf-8 -*-
"""
Training script for CGLUE-SOE-OT model using real (preprocessed)
scRNA-seq and scATAC-seq data with ordered age labels.

This script assumes the modified CGLUE-SOE-OT model files
(e.g., cglue_soe_ot.py, cglue_soe_components_ot.py) are
accessible within the scglue package structure or Python path.
"""

import sys
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
# import networkx as nx # No longer needed for training script itself
import torch

# --- Add project root to path if necessary ---
# Adjust this path if your script is located elsewhere relative to the scglue package
# project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)

try:
    # Import the modified model and config function
    import scglue
    from scglue.models.cglue_soe_ot import CGLUESOE_OT_Model, configure_dataset_cglue_soe
    # AnnDatasetWithLabels is still needed, GraphDataset is not
    from scglue.models.data import AnnDatasetWithLabels
    # from scglue.utils import config as scglue_config # Optional config import
except ImportError as e:
    print(f"Error importing modified scglue components: {e}")
    print("Please ensure the modified scglue package (with CGLUE-SOE-OT) is installed or accessible.")
    sys.exit(1)
except AttributeError as e:
    print(f"Error importing specific components (check file/class names): {e}")
    print("Ensure 'CGLUESOE_OT_Model' and 'configure_dataset_cglue_soe' exist in 'scglue.models.cglue_soe_ot'.")
    sys.exit(1)


# --- Configuration ---
SEED = 0
torch.manual_seed(SEED)
np.random.seed(SEED)
SAVE_DIR = "cglue_soe_ot_adult_human_kidney_output" # Changed save directory name
os.makedirs(SAVE_DIR, exist_ok=True)

In [20]:
# --- 1. Load Your Real Data ---
print("Loading real data...")

atac_adata= adata_ATAC
rna_adata = adata_RNA
adatas = {"rna": rna_adata, "atac": atac_adata}

# --- 2. Preprocessing Steps ---
print("Preprocessing data...")

# ** Crucial: Ensure 'age' column is ordered categorical **
# Define the correct order
age_order = ['50', '52', '54','61','62']
label_col = 'age' # MODIFY AS NEEDED
try:
    for k in adatas:
        if 'age' not in adatas[k].obs:
            raise ValueError(f"'age' column not found in adata '{k}'.")
        adatas[k].obs['age'] = pd.Categorical(
            adatas[k].obs['age'], categories=age_order, ordered=True
        )
        # Check if any values became NaN after conversion (means original values didn't match categories)
        if adatas[k].obs['age'].isnull().any():
             print(f"Warning: Some 'age' values in adata '{k}' did not match the expected categories {age_order} and resulted in NaNs.")
             # Optional: Handle NaNs, e.g., remove cells or raise error
             # adatas[k] = adatas[k][adatas[k].obs['age'].notna()].copy()
except Exception as e:
    print(f"Error processing 'age' column: {e}")
    sys.exit(1)


Loading real data...
Preprocessing data...


In [21]:
atac_adata.var['highly_variable'] = True

In [22]:

# --- 3. Define Vertices (No Graph Construction Needed) ---
# The model still needs the list of all unique features across datasets
# for the feature embedding layer.
print("Defining feature vertices...")
vertices = pd.Index(rna_adata.var_names.tolist() + atac_adata.var_names.tolist()).unique()
print(f"Total unique features (vertices): {len(vertices)}")


Defining feature vertices...
Total unique features (vertices): 17241


In [23]:

# --- 4. Configure Datasets for CGLUE-SOE-OT ---
print("Configuring datasets for CGLUE-SOE-OT...")
# ** Choose appropriate prob_model based on your preprocessing **
# 'Normal' or 'ZILN' for log-normalized RNA.
# 'Normal', 'ZILN', or 'Bernoulli' for ATAC depending on preprocessing.
try:
    configure_dataset_cglue_soe( # Use the same config function name
        adatas["rna"],
        prob_model="Normal", # Adjust if needed (e.g., "NB" if using raw counts)
        use_highly_variable=True, # Use selected HVGs
        use_label=label_col, # Use the ordered label column
        use_layer=None, # Assuming preprocessed data is in .X
        use_rep=None,   # Or specify a representation like 'X_pca' if desired
        use_obs_names=False # Data is unpaired
    )
    configure_dataset_cglue_soe( # Use the same config function name
        adatas["atac"],
        prob_model="Bernoulli", # Adjust based on ATAC preprocessing
        use_highly_variable=True, # Use selected HVGs
        use_label=label_col, # Use the ordered label column
        use_layer=None, # Assuming preprocessed data is in .X
        use_rep=None,   # Or specify a representation like 'X_lsi' if desired
        use_obs_names=False # Data is unpaired
    )
    print("Datasets configured.")
except Exception as e:
    print(f"Error configuring datasets: {e}")
    sys.exit(1)


Configuring datasets for CGLUE-SOE-OT...
[INFO] CGLUESOE_OT_Model: Dataset configured for CGLUE-SOE-OT using label 'age'.
[INFO] CGLUESOE_OT_Model: Dataset configured for CGLUE-SOE-OT using label 'age'.
Datasets configured.


In [24]:


# --- 5. Initialize CGLUE-SOE-OT Model ---
print("Initializing CGLUE-SOE-OT model...")
try:
    model = CGLUESOE_OT_Model(
        adatas=adatas,
        x2u_h_depth=2, 
        x2u_h_dim=256,                 
        u2x_h_depth=2, 
        u2x_h_dim=256, 
        dropout=0.2,
        vertices=vertices.tolist(), # Pass the combined feature list
        latent_dim=50, # Example latent dimension, adjust as needed
        random_seed=SEED
    )
    print("Model initialized.")
except Exception as e:
    print(f"Error initializing model: {e}")
    sys.exit(1)

# --- 6. Compile Model ---
print("Compiling model...")
Times = '8'
lam_kl=0.1           # Weight for KL divergence regularization 调低有利于聚类更紧密
lam_data=1.0         # Weight for data reconstruction loss 调高有利于聚类更紧密
lam_triplet=500      # Weight for triplet loss (NEEDS TUNING)
lam_ot=10.0           # Weight for Optimal Transport loss (NEEDS TUNING)
triplet_margin=3   # Margin for triplet loss (NEEDS TUNING)有可能数据本身就是按照时间序列排序的
ot_epsilon=0.1       # OT Sinkhorn regularization (NEEDS TUNING) 较小的值 (如当前的 0.1) 匹配更精确，但也可能对噪声更敏感
ot_tau=1.0          # OT Sinkhorn unbalancedness (NEEDS TUNING)
ot_max_iter=150

max_epochs=500
# ** Tune hyperparameters, especially loss weights **
try:
    model.compile(
        lr=1e-3,              # Learning rate for the single optimizer
        # lam_graph removed
        # lam_align removed
        # dsc_lr removed
        # train_ratio removed
        lam_kl=lam_kl,           # Weight for KL divergence regularization 调低有利于聚类更紧密
        lam_data=lam_data,         # Weight for data reconstruction loss 调高有利于聚类更紧密,也不能太高，不然就会有不连续的问题
        lam_triplet=lam_triplet,      # Weight for triplet loss (NEEDS TUNING)
        lam_ot=lam_ot,           # Weight for Optimal Transport loss (NEEDS TUNING)
        triplet_margin=triplet_margin,   # Margin for triplet loss (NEEDS TUNING)有可能数据本身就是按照时间序列排序的
        ot_epsilon=ot_epsilon,       # OT Sinkhorn regularization (NEEDS TUNING) 较小的值 (如当前的 0.1) 匹配更精确，但也可能对噪声更敏感
        ot_tau=ot_tau,           # OT Sinkhorn unbalancedness (NEEDS TUNING)
        ot_max_iter=ot_max_iter,      # OT Sinkhorn iterations
         #weight_decay=1e-5   # Example: Add optimizer args directly
    )
    print("Model compiled.")
except Exception as e:
    print(f"Error compiling model: {e}")
    sys.exit(1)

# --- 7. Train Model ---
print("Fitting model...")
# Fit the model (No GraphDataset needed)
# Adjust training parameters as needed
try:
    model.fit(
        adatas=adatas,            # Pass adatas dictionary
        # graph removed
        val_split=0.2,            # Use 10% of data for validation
        data_batch_size=256,      # Adjust based on memory
        # graph_batch_size removed
        max_epochs=max_epochs,           # Set a reasonable number of epochs
        patience=35,              # Early stopping patience
        reduce_lr_patience=20,    # LR scheduling patience
        wait_n_lrs=3,             # Wait for LR reductions before stopping
        num_workers=0,            # Set > 0 for parallel data loading if needed
        directory=SAVE_DIR        # Directory for logs and checkpoints
    )
    print("Model fitting finished.")
except Exception as e:
    print(f"An error occurred during model fitting: {e}")
    import traceback
    traceback.print_exc()
    sys.exit(1)




Initializing CGLUE-SOE-OT model...
[INFO] autodevice: Using GPU 3 as computation device.
Model initialized.
Compiling model...
Model compiled.
Fitting model...



For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.



[INFO] CGLUESOE_OT_Trainer: [Epoch 10] Train={'x_rna_nll': '642.8979', 'x_rna_kl': '41.3605', 'x_rna_elbo': '647.0339', 'x_atac_nll': '9790.0729', 'x_atac_kl': '46.3728', 'x_atac_elbo': '9794.7102', 'triplet_loss': '0.0431', 'ot_loss': '21.7233', 'total_loss': '10680.5436'}, Val={'x_rna_nll': '612.4531', 'x_rna_kl': '41.9420', 'x_rna_elbo': '616.6473', 'x_atac_nll': '9683.2565', 'x_atac_kl': '46.5366', 'x_atac_elbo': '9687.9104', 'triplet_loss': '0.3592', 'ot_loss': '9.0364', 'total_loss': '10574.5036'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 20] Train={'x_rna_nll': '81.4763', 'x_rna_kl': '80.4943', 'x_rna_elbo': '89.5257', 'x_atac_nll': '8533.9380', 'x_atac_kl': '85.2072', 'x_atac_elbo': '8542.4588', 'triplet_loss': '0.0307', 'ot_loss': '10.4924', 'total_loss': '8752.2472'}, Val={'x_rna_nll': '51.8669', 'x_rna_kl': '81.2970', 'x_rna_elbo': '59.9966', 'x_atac_nll': '8532.3712', 'x_atac_kl': '85.2515', 'x_atac_elbo': '8540.8964', 'triplet_loss': '0.2840', 'ot_loss': '5.0905', 'total_loss': '

[INFO] LRScheduler: Learning rate reduction: step 5
[INFO] CGLUESOE_OT_Trainer: [Epoch 210] Train={'x_rna_nll': '-2610.2561', 'x_rna_kl': '137.1647', 'x_rna_elbo': '-2596.5396', 'x_atac_nll': '7691.0436', 'x_atac_kl': '158.3074', 'x_atac_elbo': '7706.8744', 'triplet_loss': '0.0021', 'ot_loss': '1.4177', 'total_loss': '5125.5606'}, Val={'x_rna_nll': '-2358.3988', 'x_rna_kl': '135.1153', 'x_rna_elbo': '-2344.8872', 'x_atac_nll': '7795.1758', 'x_atac_kl': '155.7877', 'x_atac_elbo': '7810.7546', 'triplet_loss': '0.1595', 'ot_loss': '1.7452', 'total_loss': '5563.0871'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 220] Train={'x_rna_nll': '-2611.7916', 'x_rna_kl': '137.2496', 'x_rna_elbo': '-2598.0666', 'x_atac_nll': '7688.7067', 'x_atac_kl': '158.6958', 'x_atac_elbo': '7704.5763', 'triplet_loss': '0.0015', 'ot_loss': '1.3681', 'total_loss': '5120.9211'}, Val={'x_rna_nll': '-2358.7906', 'x_rna_kl': '135.2065', 'x_rna_elbo': '-2345.2699', 'x_atac_nll': '7795.1821', 'x_atac_kl': '155.4252', 'x_atac_elbo

[INFO] CGLUESOE_OT_Trainer: [Epoch 370] Train={'x_rna_nll': '-2625.1822', 'x_rna_kl': '137.1432', 'x_rna_elbo': '-2611.4679', 'x_atac_nll': '7674.3845', 'x_atac_kl': '158.3904', 'x_atac_elbo': '7690.2235', 'triplet_loss': '0.0007', 'ot_loss': '1.3968', 'total_loss': '5093.0864'}, Val={'x_rna_nll': '-2358.4152', 'x_rna_kl': '135.1757', 'x_rna_elbo': '-2344.8976', 'x_atac_nll': '7795.1216', 'x_atac_kl': '155.6963', 'x_atac_elbo': '7810.6913', 'triplet_loss': '0.1603', 'ot_loss': '1.7743', 'total_loss': '5563.6749'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 380] Train={'x_rna_nll': '-2614.4993', 'x_rna_kl': '137.1267', 'x_rna_elbo': '-2600.7866', 'x_atac_nll': '7687.7285', 'x_atac_kl': '158.3073', 'x_atac_elbo': '7703.5593', 'triplet_loss': '0.0028', 'ot_loss': '1.4240', 'total_loss': '5118.4279'}, Val={'x_rna_nll': '-2358.4345', 'x_rna_kl': '134.9345', 'x_rna_elbo': '-2344.9411', 'x_atac_nll': '7795.4269', 'x_atac_kl': '155.9225', 'x_atac_elbo': '7811.0191', 'triplet_loss': '0.1630', 'ot_loss':

In [25]:
# --- 8. Post-Training Analysis ---
print("Extracting embeddings...")
try:
    # Use a different key for the new model's embeddings
    embedding_key = "X_cgluesoe_ot"
    rna_adata.obsm[embedding_key] = model.encode_data("rna", rna_adata)
    atac_adata.obsm[embedding_key] = model.encode_data("atac", atac_adata)

    # Get feature embeddings from the nn.Embedding layer
    feature_embeddings = model.get_feature_embeddings()
    feature_embeddings_df = pd.DataFrame(feature_embeddings, index=model.vertices)

    print("RNA embeddings shape:", rna_adata.obsm[embedding_key].shape)
    print("ATAC embeddings shape:", atac_adata.obsm[embedding_key].shape)
    print("Feature embeddings shape:", feature_embeddings_df.shape)

    # --- Optional: Save model and embeddings ---
    final_model_path = os.path.join(SAVE_DIR, "cglue_soe_ot_final_model.dill")
    print(f"Saving final model to {final_model_path}")
    model.save(final_model_path)

    rna_save_path = os.path.join(SAVE_DIR, "rna_adata_with_embeddings.h5ad")
    atac_save_path = os.path.join(SAVE_DIR, "atac_adata_with_embeddings.h5ad")
    feat_save_path = os.path.join(SAVE_DIR, "feature_embeddings.csv")

    # print(f"Saving RNA AnnData to {rna_save_path}")
    # rna_adata.write_h5ad(rna_save_path)
    # print(f"Saving ATAC AnnData to {atac_save_path}")
    # atac_adata.write_h5ad(atac_save_path)
    # print(f"Saving feature embeddings to {feat_save_path}")
    # feature_embeddings_df.to_csv(feat_save_path)


    # --- Optional: Visualization (UMAP) ---
    print("Generating UMAP visualization...")
    # Combine embeddings and obs for visualization
    all_embeddings = np.concatenate([rna_adata.obsm[embedding_key], atac_adata.obsm[embedding_key]], axis=0)
    all_obs = pd.concat([
        rna_adata.obs[[label_col]].assign(modality='rna'), # Use label_col
        atac_adata.obs[[label_col]].assign(modality='atac') # Use label_col
    ], axis=0)
    # Create combined obs_names for the visualization AnnData
    vis_obs_names = rna_adata.obs_names.tolist() + atac_adata.obs_names.tolist()
    if len(vis_obs_names) != len(all_obs): # Basic check
         print(f"Warning: Length mismatch between obs_names ({len(vis_obs_names)}) and combined obs ({len(all_obs)}). Using default index.")
         vis_obs_names = pd.RangeIndex(start=0, stop=len(all_obs), step=1)


    vis_adata = ad.AnnData(all_embeddings, dtype=all_embeddings.dtype) # Ensure dtype is set
    vis_adata.obs = all_obs
    vis_adata.obs_names = vis_obs_names # Assign combined names
    vis_adata.obsm['X_cgluesoe_ot'] = all_embeddings # Store for scanpy functions

    print("Running neighbors and UMAP...")
    sc.pp.neighbors(vis_adata, use_rep='X_cgluesoe_ot', n_neighbors=15, random_state=SEED)
    sc.tl.umap(vis_adata, min_dist=0.3, random_state=SEED) # Adjust min_dist for spread

    print("Plotting UMAP...")
    umap_save_path = f'_cglue_soe_ot_adult_human_kidney_kidney_loop_of_Henle_results_{Times}.png' # Updated filename
    sc.pl.umap(
        vis_adata,
        color=['modality', label_col], # Color by label_col
        title=['UMAP by Modality', f'UMAP by {label_col.capitalize()}'],
        save=umap_save_path,
        show=False # Set to True to display interactively if possible
    )
    print(f"UMAP plots saved to figures/ directory ({umap_save_path}).")

except Exception as e:
    print(f"An error occurred during post-training analysis: {e}")
    import traceback
    traceback.print_exc()

print("Script finished.")

Extracting embeddings...
RNA embeddings shape: (4435, 50)
ATAC embeddings shape: (7775, 50)
Feature embeddings shape: (17241, 50)
Saving final model to cglue_soe_ot_adult_human_kidney_output/cglue_soe_ot_final_model.dill
[INFO] CGLUESOE_OT_Model: Model state saved to cglue_soe_ot_adult_human_kidney_output/cglue_soe_ot_final_model.dill
Generating UMAP visualization...
Running neighbors and UMAP...


Plotting UMAP...
UMAP plots saved to figures/ directory (_cglue_soe_ot_adult_human_kidney_kidney_loop_of_Henle_results_8.png).
Script finished.


In [26]:
print("Plotting loss curves...")
try:
    # Define where to save the plot
    loss_plot_path = os.path.join(SAVE_DIR, f"cglue_soe_ot_loss_curves_adult_human_kidney_kidney_loop_of_Henle_{Times}.png")
    # Call the plotting method
    model.plot_loss_curves(save_path=loss_plot_path)
    print(f"Loss curves saved to {loss_plot_path}")
except Exception as e:
    print(f"An error occurred during loss plotting: {e}")
    import traceback
    traceback.print_exc()


Plotting loss curves...
[INFO] CGLUESOE_OT_Model: Loss curves saved to cglue_soe_ot_adult_human_kidney_output/cglue_soe_ot_loss_curves_adult_human_kidney_kidney_loop_of_Henle_8.png
Loss curves saved to cglue_soe_ot_adult_human_kidney_output/cglue_soe_ot_loss_curves_adult_human_kidney_kidney_loop_of_Henle_8.png


In [27]:
import csv
import pickle
hyperparameters = {
    "lam_kl": lam_kl,
    "lam_data": lam_data,
    "lam_triplet": lam_triplet,
    "lam_ot": lam_ot,
    "triplet_margin": triplet_margin,
    "ot_epsilon": ot_epsilon,
    "ot_tau": ot_tau,
    "ot_max_iter": ot_max_iter,
    "max_epochs": max_epochs
}

csv_filename = os.path.join(SAVE_DIR, "hyperparameters_kidney_loop_of_Henle.csv")

try:
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([f'ParameterName_{Times}', 'Value'])
        for name, value in hyperparameters.items():
            writer.writerow([name, value])
    print(f"超参数已成功保存到 {csv_filename}")


超参数已成功保存到 cglue_soe_ot_adult_human_kidney_output/hyperparameters_kidney_loop_of_Henle.csv
